# Using manual and more robust way

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
data = pd.read_csv("Audiobooks_data.csv", header=None)
df = data.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,873,2160.0,2160,10.13,10.13,0,8.91,0.0,0.0,0,0,1
1,611,1404.0,2808,6.66,13.33,1,6.50,0.0,0.0,0,182,1
2,705,324.0,324,10.13,10.13,1,9.00,0.0,0.0,1,334,1
3,391,1620.0,1620,15.31,15.31,0,9.00,0.0,0.0,0,183,1
4,819,432.0,1296,7.11,21.33,1,9.00,0.0,0.0,0,0,1


In [3]:
df.shape

(14084, 12)

In [4]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [5]:
df.dtypes

0       int64
1     float64
2       int64
3     float64
4     float64
5       int64
6     float64
7     float64
8     float64
9       int64
10      int64
11      int64
dtype: object

In [6]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000,14084.000000
mean,16772.491551,1591.281685,1678.608634,7.103791,7.543805,0.160750,8.909795,0.125659,118.586745,0.070222,61.935033,0.158833
std,9691.807248,504.340663,654.838599,4.931673,5.560129,0.367313,0.643406,0.241206,268.731935,0.472157,88.207634,0.365533
min,2.000000,216.000000,216.000000,3.860000,3.860000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8368.000000,1188.000000,1188.000000,5.330000,5.330000,0.000000,8.910000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16711.500000,1620.000000,1620.000000,5.950000,6.070000,0.000000,8.910000,0.000000,0.000000,0.000000,11.000000,0.000000
75%,25187.250000,2160.000000,2160.000000,8.000000,8.000000,0.000000,8.910000,0.130000,64.800000,0.000000,105.000000,0.000000
max,33683.000000,2160.000000,7020.000000,130.940000,130.940000,1.000000,10.000000,1.000000,2116.800000,30.000000,464.000000,1.000000


In [11]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

In [12]:
print(X.shape,y.shape)

(14084, 10) (14084,)


In [13]:
X.head()

,1,2,3,4,5,6,7,8,9,10
0,2160.0,2160,10.13,10.13,0,8.91,0.0,0.0,0,0
1,1404.0,2808,6.66,13.33,1,6.50,0.0,0.0,0,182
2,324.0,324,10.13,10.13,1,9.00,0.0,0.0,1,334
3,1620.0,1620,15.31,15.31,0,9.00,0.0,0.0,0,183
4,432.0,1296,7.11,21.33,1,9.00,0.0,0.0,0,0


In [14]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: 11, dtype: int64

In [17]:
ones_df = df[df.iloc[:,-1] == 1]
zeros_df = df[df.iloc[:,-1] == 0].iloc[:len(ones_df)]
balanced_df = pd.concat([ones_df,zeros_df]).sample(frac=1).reset_index(drop=True)

In [20]:
ones_df.shape

(2237, 12)

In [18]:
balanced_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,698,648.0,648,12.27,12.27,0,8.91,0.00,0.00,0,0,1
1,23437,2160.0,2160,7.30,7.30,0,8.91,0.00,669.60,0,0,0
2,18909,1620.0,1620,5.63,5.63,0,8.91,0.58,410.40,1,3,0
3,25383,1620.0,1620,7.99,7.99,1,10.00,0.40,736.56,0,10,0
4,5087,2160.0,2160,7.99,7.99,0,8.91,0.00,0.00,0,0,1


In [19]:
balanced_df.shape

(4474, 12)

In [22]:
X = balanced_df.iloc[:,1:-1]
y = balanced_df.iloc[:,-1]
print(X.shape,y.shape)

(4474, 10) (4474,)


balancing ---> scaling ---> split ---> modeling

In [23]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled,y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [27]:
print(f"Train data: {X_train.shape} and {y_train.shape}, no of 1s: {sum(y_train)}")
print(f"Test data: {X_test.shape} and {y_test.shape}, no of 1s: {sum(y_test)}")
print(f"Validation data: {X_val.shape} and {y_val.shape}, no of 1s: {sum(y_val)}")


Train data: (3579, 10) and (3579,), no of 1s: 1790
Test data: (448, 10) and (448,), no of 1s: 224
Validation data: (447, 10) and (447,), no of 1s: 223


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification: 1 output neuron with sigmoid
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=50,
          batch_size=32,
          callbacks=[early_stop])


Epoch 1/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7858 - loss: 0.5432 - val_accuracy: 0.8814 - val_loss: 0.3206
Epoch 2/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8840 - loss: 0.3076 - val_accuracy: 0.8926 - val_loss: 0.2882
Epoch 3/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8920 - loss: 0.2836 - val_accuracy: 0.9016 - val_loss: 0.2746
Epoch 4/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8936 - loss: 0.2679 - val_accuracy: 0.9016 - val_loss: 0.2667
Epoch 5/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9010 - loss: 0.2584 - val_accuracy: 0.9016 - val_loss: 0.2659
Epoch 6/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8965 - loss: 0.2606 - val_accuracy: 0.9038 - val_loss: 0.2595
Epoch 7/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9035 - loss: 0.2479 - val_accuracy: 0.8993 - val_loss: 0.2699
Epoch 8/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9082 - loss: 0.2433 - val_accuracy: 0.

In [29]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.2%}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9233 - loss: 0.2069
Test accuracy: 91.52%


data exploration --> balancing --> scaling ---> data split --> define model --> compile model -->
train model --> evaluate model --> hypertuning

In [ ]:
never do scaling after scaling.